# Extract

- Import libraries

In [ ]:
import time
import json
import requests
from kafka import KafkaProducer
from datetime import datetime
import logging

- Define functions

In [ ]:
# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Binance API endpoint for symbol price
BINANCE_API_URL = "https://api.binance.com/api/v3/ticker/price"
SYMBOL = "BTCUSDT"
KAFKA_TOPIC = "btc-price"
KAFKA_BOOTSTRAP_SERVERS = ['localhost:9092']

def create_kafka_producer():
    try:
        producer = KafkaProducer(
            bootstrap_servers=KAFKA_BOOTSTRAP_SERVERS,
            value_serializer=lambda v: json.dumps(v).encode('utf-8'),
            retries=5,  # Thử lại 5 lần nếu gặp lỗi
            retry_backoff_ms=500,  # Đợi 500ms giữa các lần thử lại
            api_version=(0, 10, 1)  # Đảm bảo tương thích với Kafka 3.9.0
        )
        logger.info("Kafka producer created successfully")
        return producer
    except Exception as e:
        logger.error(f"Failed to create Kafka producer: {e}")
        raise

def fetch_binance_price():
    try:
        params = {'symbol': SYMBOL}
        response = requests.get(BINANCE_API_URL, params=params, timeout=5)
        response.raise_for_status()
        data = response.json()
        
        # Validate JSON format
        if not isinstance(data.get('symbol'), str) or not isinstance(data.get('price'), (str, float)):
            raise ValueError("Invalid JSON format from Binance API")
        
        # Convert price to float
        price = float(data['price'])
        
        return {
            'symbol': data['symbol'],
            'price': price
        }
    except Exception as e:
        logger.error(f"Error fetching price from Binance: {e}")
        return None

def main():
    try:
        producer = create_kafka_producer()
        interval = 0.1  # 100 milliseconds
        
        while True:
            start_time = time.time()
            
            # Fetch price data
            price_data = fetch_binance_price()
            
            if price_data:
                # Add event-time timestamp (ISO8601 format)
                event_time = datetime.now().isoformat()
                message = {
                    'symbol': price_data['symbol'],
                    'price': price_data['price'],
                    'timestamp': event_time
                }
                
                # Send to Kafka
                try:
                    producer.send(KAFKA_TOPIC, message)
                    producer.flush()  # Đảm bảo tin nhắn được gửi ngay lập tức
                    logger.info(f"Sent message: {message}")
                except Exception as e:
                    logger.error(f"Error sending to Kafka: {e}")
            
            # Maintain frequency (100ms)
            elapsed = time.time() - start_time
            if elapsed < interval:
                time.sleep(interval - elapsed)
                
    except Exception as e:
        logger.error(f"Main loop failed: {e}")
        raise
    finally:
        logger.info("Closing Kafka producer")
        if 'producer' in locals():
            producer.close()

- Main

In [ ]:
if __name__ == "__main__":
    try:
        main()
    except KeyboardInterrupt:
        logger.info("Shutting down producer")
    except Exception as e:
        logger.error(f"Producer failed: {e}")